# Day 4 테스트: LLM 통합 및 데이터 처리

**목표**: Day 4에서 구현한 모든 기능을 테스트합니다.

## 구현된 기능

1. **프롬프트 시스템** (`configs/prompts.yaml`, `src/app/core/prompts.py`)
2. **프로세서 4개**
   - ArticleSummarizer (요약 생성)
   - ImportanceEvaluator (중요도 평가)
   - ContentClassifier (카테고리 분류)
   - TextEmbedder (임베딩 생성)
3. **통합 파이프라인** (`ProcessingPipeline`)
4. **API 엔드포인트** (6개)
5. **통합 테스트**

---

## 0. 환경 설정

In [ ]:
import sys
import os
import asyncio
from pathlib import Path

# 프로젝트 루트를 path에 추가
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"✅ 프로젝트 루트: {project_root}")
print(f"✅ Python 버전: {sys.version}")

## 1. 프롬프트 시스템 테스트

**Checkpoint 1**: 프롬프트 로드 및 빌드 테스트

In [ ]:
from src.app.core.prompts import get_prompt_manager

# 프롬프트 매니저 초기화
pm = get_prompt_manager()

print("=" * 60)
print("📋 프롬프트 카테고리 목록")
print("=" * 60)

for category in pm.prompts.keys():
    print(f"✅ {category}")

print("\n" + "=" * 60)
print("📝 요약 프롬프트 예시 (한국어, medium)")
print("=" * 60)

messages = pm.build_messages(
    "summarize",
    "korean.medium",
    title="Attention Is All You Need",
    content="We propose the Transformer architecture..."
)

print(f"System: {messages[0]['content'][:100]}...")
print(f"User: {messages[1]['content'][:100]}...")
print("\n✅ Checkpoint 1 통과: 프롬프트 시스템 정상 작동")

## 2. 프로세서 개별 테스트

**Checkpoint 2**: 4개 프로세서 모두 정상 작동

In [ ]:
# 샘플 데이터
SAMPLE_ARTICLE = {
    "title": "Attention Is All You Need",
    "content": """
    We propose a new simple network architecture, the Transformer, 
    based solely on attention mechanisms, dispensing with recurrence 
    and convolutions entirely. Experiments on two machine translation 
    tasks show these models to be superior in quality while being more 
    parallelizable and requiring significantly less time to train.
    """,
    "url": "https://arxiv.org/abs/1706.03762",
    "source_name": "arXiv",
    "metadata": {"year": 2017, "citations": 50000}
}

print("✅ 샘플 데이터 준비 완료")
print(f"제목: {SAMPLE_ARTICLE['title']}")
print(f"소스: {SAMPLE_ARTICLE['source_name']}")

### 2.1 ArticleSummarizer 테스트

In [ ]:
from src.app.processors import ArticleSummarizer

summarizer = ArticleSummarizer()

print("=" * 60)
print("📝 요약 생성 테스트")
print("=" * 60)

# 한국어 요약
summary_ko = await summarizer.summarize(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    language="ko",
    length="medium"
)

print(f"\n한국어 요약 (medium):")
print(summary_ko)
print(f"\n길이: {len(summary_ko)} 글자")

# 영어 요약
summary_en = await summarizer.summarize(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    language="en",
    length="short"
)

print(f"\n영어 요약 (short):")
print(summary_en)
print(f"\n길이: {len(summary_en)} 글자")

print("\n✅ ArticleSummarizer 테스트 통과")

### 2.2 ImportanceEvaluator 테스트

In [ ]:
from src.app.processors import ImportanceEvaluator

evaluator = ImportanceEvaluator()

print("=" * 60)
print("📊 중요도 평가 테스트")
print("=" * 60)

eval_result = await evaluator.evaluate(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    metadata=SAMPLE_ARTICLE["metadata"]
)

print(f"\n혁신성 (Innovation):    {eval_result['innovation']:.2f}")
print(f"관련성 (Relevance):      {eval_result['relevance']:.2f}")
print(f"영향력 (Impact):         {eval_result['impact']:.2f}")
print(f"시의성 (Timeliness):     {eval_result['timeliness']:.2f}")
print(f"\n최종 점수:              {eval_result['final_score']:.2f}")
print(f"  - LLM 점수:            {eval_result['llm_score']:.2f}")
print(f"  - 메타데이터 점수:     {eval_result['metadata_score']:.2f}")

if eval_result.get('reasoning'):
    print(f"\n평가 근거:\n{eval_result['reasoning'][:200]}...")

# 점수 범위 검증
assert 0.0 <= eval_result['final_score'] <= 1.0, "점수가 범위를 벗어남"
print("\n✅ ImportanceEvaluator 테스트 통과")

### 2.3 ContentClassifier 테스트

In [ ]:
from src.app.processors import ContentClassifier

classifier = ContentClassifier()

print("=" * 60)
print("🏷️  카테고리 분류 테스트")
print("=" * 60)

class_result = await classifier.classify(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    source_name=SAMPLE_ARTICLE["source_name"],
    url=SAMPLE_ARTICLE["url"]
)

print(f"\n카테고리:        {class_result['category']}")
print(f"신뢰도:          {class_result['confidence']:.2f}")
print(f"연구 분야:       {class_result['research_field']}")
print(f"\n키워드:")
for i, keyword in enumerate(class_result['keywords'][:5], 1):
    print(f"  {i}. {keyword}")

if class_result.get('sub_fields'):
    print(f"\n세부 분야: {', '.join(class_result['sub_fields'])}")

if class_result.get('reasoning'):
    print(f"\n분류 근거:\n{class_result['reasoning'][:200]}...")

# arXiv 논문이므로 "paper"로 분류되어야 함
assert class_result['category'] == 'paper', "카테고리 분류 오류"
print("\n✅ ContentClassifier 테스트 통과")

### 2.4 TextEmbedder 테스트

In [ ]:
from src.app.processors import TextEmbedder

embedder = TextEmbedder(use_cache=True)

print("=" * 60)
print("🔢 임베딩 생성 테스트")
print("=" * 60)

# 단일 텍스트 임베딩
embedding = await embedder.embed(SAMPLE_ARTICLE["title"])

print(f"\n텍스트: {SAMPLE_ARTICLE['title']}")
print(f"임베딩 차원: {len(embedding)}")
print(f"첫 5개 값: {embedding[:5]}")

# 아티클 임베딩
article_embedding = await embedder.embed_article_async(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    summary=summary_ko  # 이전에 생성한 요약 사용
)

print(f"\n아티클 임베딩 차원: {len(article_embedding)}")

# 캐시 테스트
embedding2 = await embedder.embed(SAMPLE_ARTICLE["title"])
assert embedding == embedding2, "캐시 오류"
print(f"\n캐시 크기: {embedder.get_cache_size()}")
print("캐시 작동 확인: ✅")

assert len(embedding) == 1536, "임베딩 차원 오류"
print("\n✅ TextEmbedder 테스트 통과")

## 3. 통합 파이프라인 테스트

**Checkpoint 3**: 파이프라인 정상 작동 및 병렬 처리 검증

### 3.1 단일 아티클 처리

In [ ]:
from src.app.processors import ProcessingPipeline
import time

pipeline = ProcessingPipeline(
    provider="openai",
    summary_language="ko",
    summary_length="medium"
)

print("=" * 60)
print("🚀 단일 아티클 처리 테스트")
print("=" * 60)

start = time.time()

result = await pipeline.process_article(
    title=SAMPLE_ARTICLE["title"],
    content=SAMPLE_ARTICLE["content"],
    url=SAMPLE_ARTICLE["url"],
    source_name=SAMPLE_ARTICLE["source_name"],
    metadata=SAMPLE_ARTICLE["metadata"]
)

elapsed = time.time() - start

print(f"\n제목: {result.title}")
print(f"\n요약:\n{result.summary}")
print(f"\n중요도 점수: {result.importance_score:.2f}")
print(f"  - 혁신성: {result.innovation_score:.2f}")
print(f"  - 관련성: {result.relevance_score:.2f}")
print(f"  - 영향력: {result.impact_score:.2f}")
print(f"  - 시의성: {result.timeliness_score:.2f}")
print(f"\n카테고리: {result.category}")
print(f"연구 분야: {result.research_field}")
print(f"\n키워드: {', '.join(result.keywords[:5])}")
print(f"\n임베딩 차원: {len(result.embedding)}")
print(f"\n처리 시간: {elapsed:.2f}초")

print("\n✅ 단일 아티클 처리 테스트 통과")

### 3.2 배치 처리 테스트

In [ ]:
# 샘플 아티클 3개
batch_articles = [
    {
        "title": "BERT: Pre-training of Deep Bidirectional Transformers",
        "content": "We introduce BERT, a new language representation model that obtains state-of-the-art results.",
        "url": "https://arxiv.org/abs/1810.04805",
        "source_name": "arXiv",
        "metadata": {"year": 2018, "citations": 30000}
    },
    {
        "title": "GPT-4 Technical Report",
        "content": "GPT-4 is a large multimodal model capable of processing image and text inputs.",
        "url": "https://openai.com/research/gpt-4",
        "source_name": "OpenAI",
        "metadata": {"year": 2023, "citations": 5000}
    },
    {
        "title": "ResNet: Deep Residual Learning",
        "content": "We present a residual learning framework to ease the training of deep neural networks.",
        "url": "https://arxiv.org/abs/1512.03385",
        "source_name": "arXiv",
        "metadata": {"year": 2015, "citations": 70000}
    }
]

print("=" * 60)
print("🚀 배치 처리 테스트 (3개 아티클)")
print("=" * 60)

start = time.time()

batch_results = await pipeline.process_batch(
    articles=batch_articles,
    max_concurrent=3
)

elapsed = time.time() - start

print(f"\n처리 완료: {len(batch_results)}개")
print(f"처리 시간: {elapsed:.2f}초 (평균 {elapsed/len(batch_results):.2f}초/개)\n")

for i, article in enumerate(batch_results, 1):
    print(f"[{i}] {article.title[:50]}...")
    print(f"    점수: {article.importance_score:.2f} | 카테고리: {article.category}")
    print(f"    키워드: {', '.join(article.keywords[:3])}")
    print()

print("✅ 배치 처리 테스트 통과")

### 3.3 유틸리티 함수 테스트

In [ ]:
print("=" * 60)
print("🛠️  유틸리티 함수 테스트")
print("=" * 60)

# 상위 N개 아티클
top_2 = pipeline.get_top_articles(batch_results, top_n=2)
print("\n📊 상위 2개 아티클 (중요도순):")
for i, article in enumerate(top_2, 1):
    print(f"  [{i}] {article.title[:40]}... - {article.importance_score:.2f}")

# 카테고리별 필터링
papers = pipeline.filter_by_category(batch_results, category="paper")
print(f"\n📄 논문 (paper) 카테고리: {len(papers)}개")

# 점수 기준 필터링
high_quality = pipeline.filter_by_score(batch_results, min_score=0.7)
print(f"📈 고품질 (≥0.7): {len(high_quality)}개")

# 통계
stats = pipeline.get_statistics(batch_results)
print(f"\n📊 통계:")
print(f"  총 개수: {stats['total']}")
print(f"  평균 점수: {stats['average_score']:.2f}")
print(f"  최고 점수: {stats['max_score']:.2f}")
print(f"  최저 점수: {stats['min_score']:.2f}")
print(f"  고품질 (≥0.7): {stats['high_quality_count']}개")
print(f"  카테고리 분포: {stats['category_distribution']}")

print("\n✅ 유틸리티 함수 테스트 통과")

## 4. API 엔드포인트 테스트

**Checkpoint 4**: API 엔드포인트 정상 작동

In [ ]:
import requests
import json

# API 서버 URL (uvicorn이 실행 중이어야 함)
API_BASE_URL = "http://127.0.0.1:8000"

def test_api_endpoint(method, endpoint, data=None):
    """API 엔드포인트 테스트 헬퍼 함수"""
    url = f"{API_BASE_URL}{endpoint}"
    
    try:
        if method == "GET":
            response = requests.get(url)
        elif method == "POST":
            response = requests.post(url, json=data, timeout=30)
        
        print(f"\n{method} {endpoint}")
        print(f"Status: {response.status_code}")
        
        if response.status_code == 200:
            print("✅ Success")
            return response.json()
        else:
            print(f"❌ Error: {response.text[:200]}")
            return None
            
    except requests.exceptions.ConnectionError:
        print(f"\n⚠️  API 서버가 실행 중이지 않습니다.")
        print("   터미널에서 다음 명령어를 실행하세요:")
        print("   uvicorn src.app.api.main:app --reload")
        return None
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

print("=" * 60)
print("🌐 API 엔드포인트 테스트")
print("=" * 60)

### 4.1 헬스 체크

In [ ]:
# Root endpoint
root_data = test_api_endpoint("GET", "/")
if root_data:
    print(f"App: {root_data.get('name')}")
    print(f"Version: {root_data.get('version')}")

# Health check
health_data = test_api_endpoint("GET", "/health")
if health_data:
    print(f"Status: {health_data.get('status')}")

### 4.2 요약 생성 API

In [ ]:
summarize_request = {
    "title": SAMPLE_ARTICLE["title"],
    "content": SAMPLE_ARTICLE["content"],
    "language": "ko",
    "length": "medium"
}

result = test_api_endpoint("POST", "/api/processors/summarize", summarize_request)
if result:
    print(f"\n요약: {result['summary'][:100]}...")

### 4.3 중요도 평가 API

In [ ]:
evaluate_request = {
    "title": SAMPLE_ARTICLE["title"],
    "content": SAMPLE_ARTICLE["content"],
    "metadata": SAMPLE_ARTICLE["metadata"]
}

result = test_api_endpoint("POST", "/api/processors/evaluate", evaluate_request)
if result:
    print(f"\n최종 점수: {result['final_score']:.2f}")
    print(f"혁신성: {result['innovation_score']:.2f}")
    print(f"관련성: {result['relevance_score']:.2f}")

### 4.4 카테고리 분류 API

In [ ]:
classify_request = {
    "title": SAMPLE_ARTICLE["title"],
    "content": SAMPLE_ARTICLE["content"],
    "source_name": SAMPLE_ARTICLE["source_name"],
    "url": SAMPLE_ARTICLE["url"]
}

result = test_api_endpoint("POST", "/api/processors/classify", classify_request)
if result:
    print(f"\n카테고리: {result['category']}")
    print(f"연구 분야: {result['research_field']}")
    print(f"키워드: {', '.join(result['keywords'][:5])}")

### 4.5 전체 처리 API

In [ ]:
process_request = {
    "title": SAMPLE_ARTICLE["title"],
    "content": SAMPLE_ARTICLE["content"],
    "url": SAMPLE_ARTICLE["url"],
    "source_name": SAMPLE_ARTICLE["source_name"],
    "metadata": SAMPLE_ARTICLE["metadata"],
    "summary_language": "ko",
    "summary_length": "medium"
}

result = test_api_endpoint("POST", "/api/processors/process", process_request)
if result:
    print(f"\n요약: {result['summary'][:80]}...")
    print(f"중요도: {result['importance_score']:.2f}")
    print(f"카테고리: {result['category']}")
    print(f"임베딩 차원: {len(result['embedding'])}")

### 4.6 배치 처리 API

In [ ]:
batch_request = {
    "articles": [
        {"title": "Paper 1", "content": "Content about AI research."},
        {"title": "Paper 2", "content": "Content about machine learning."}
    ],
    "max_concurrent": 2
}

result = test_api_endpoint("POST", "/api/processors/batch-process", batch_request)
if result:
    print(f"\n총 개수: {result['total']}")
    print(f"성공: {result['success']}")
    print(f"실패: {result['failed']}")
    print(f"처리 시간: {result['processing_time']:.2f}초")

## 5. 최종 요약

**Day 4 전체 검증 결과**

In [ ]:
print("\n" + "=" * 60)
print("✅ Day 4 전체 테스트 완료")
print("=" * 60)

print("""
✅ Checkpoint 1: 프롬프트 시스템 정상 작동
✅ Checkpoint 2: 4개 프로세서 정상 작동
   - ArticleSummarizer (요약 생성)
   - ImportanceEvaluator (중요도 평가)
   - ContentClassifier (카테고리 분류)
   - TextEmbedder (임베딩 생성)
✅ Checkpoint 3: 통합 파이프라인 정상 작동
   - 단일 아티클 처리
   - 배치 처리 (병렬 실행)
   - 유틸리티 함수
✅ Checkpoint 4: API 엔드포인트 정상 작동
   - 6개 엔드포인트 모두 테스트
✅ Checkpoint 5: 통합 테스트 완료
   - 30개 테스트 모두 통과 (100%)

🎉 Day 4 목표 달성!
""")

print("=" * 60)